# 📓 The GenAI Revolution Cookbook

**Title:** How to Build Multi-Agent AI Systems with CrewAI and YAML

**Description:** Build production-ready multi-agent AI systems with CrewAI using reusable YAML-first patterns, explicit tools and tasks, guardrails, and interactive training loops.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## Introduction

This guide walks you through building a multi\-agent customer feedback analysis system with CrewAI. You will define three specialized agents—one for sentiment evaluation, one for report summarization, and one for visualization—that collaborate in a structured workflow to process raw feedback, generate insights, and produce a final report with charts.

You will see how to set up agents and tasks in YAML, integrate tools (in this case a CSV\-file reader) for grounded analysis, validate structured outputs, and produce visualization snippets. All code is ready to run (e.g. in Colab) with minimal setup.

By the end, you will have a functioning multi\-agent system that converts raw feedback data into reusable, structured analytical reports.

---

## Building blocks of multi\-agent systems in CrewAI

When you build a multi\-agent system, think of it like structuring a team. In CrewAI you organize around **Agents**, **Tasks**, **Tools**, **Crews**, and **Training**. These are key to scaling and maintaining the system. CrewAI supports definition of agents and tasks in YAML, separating configuration from code. This makes things easier to manage and adjust over time. ([thegenairevolution.com](https://thegenairevolution.com/building-multi-agent-ai-systems-a-practical-guide-with-crewai/))

### Agents

Agents represent roles, goals, and backstories. You define them in YAML. Each agent is assigned tools (if needed), and memory or code permissions depending on their responsibilities. That keeps responsibilities explicit and helps reduce errors like hallucinations. ([thegenairevolution.com](https://thegenairevolution.com/building-multi-agent-ai-systems-a-practical-guide-with-crewai/))

If you’re interested in how agent design can be extended to more complex, stateful workflows, our guide on [building a stateful AI agent with LangGraph](https://thegenairevolution.com/blog/44830763/how-to-build-a-stateful-ai-agent-with-langgraph-step-by-step-5) offers a practical, step\-by\-step approach.

### Tasks

Tasks are what you want agents to do. Each task has:

* A description of what needs to be done
* An expected output format
* An agent assignment

You define tasks separately in YAML. You can have more tasks than agents. One agent may handle multiple tasks. ([thegenairevolution.com](https://thegenairevolution.com/building-multi-agent-ai-systems-a-practical-guide-with-crewai/))

For a deeper dive into extracting structured information from unstructured data, see our article on [structured data extraction with LLMs](https://thegenairevolution.com/blog/44830763/structured-data-extraction-with-llms-how-to-build-a-pipeline-3), which explores building robust pipelines for production use.

### Tools

Tools extend what agents can do. For customer feedback analysis, you will want tools to read CSV files so agents can reference concrete feedback entries. CrewAI has built\-in tools and you can add custom ones. ([thegenairevolution.com](https://thegenairevolution.com/building-multi-agent-ai-systems-a-practical-guide-with-crewai/))

If you’re curious about how to optimize tool use and manage data access for agents, the [Model Context Protocol (MCP) guide](https://thegenairevolution.com/blog/44830763/model-context-protocol-mcp-explained-2025-guide-for-builders) explains how to standardize tool and data access for reliable, auditable AI workflows.

### Crews

A Crew ties agents and tasks together into a workflow. You decide how agents collaborate. CrewAI supports several process types including sequential workflows. Sequential means one task follows another. That gives you clarity in dependencies and results. ([thegenairevolution.com](https://thegenairevolution.com/building-multi-agent-ai-systems-a-practical-guide-with-crewai/))

### Training

You may not get perfect results from the start. CrewAI supports training mode. You can run agents, pause interactions, give feedback, correct mistakes. Agents learn iteratively. This helps adapt the system to your specific data and preferences. ([thegenairevolution.com](https://thegenairevolution.com/building-multi-agent-ai-systems-a-practical-guide-with-crewai/))

---

## Creating a multi\-agent customer feedback analysis system

Let’s build a system that takes customer feedback data, analyzes sentiment and themes, creates summary tables, makes visualizations, and produces a final Markdown report.

### Installing Requirements

In [ ]:
!pip install 'crewai[tools]'

Import modules and load environment variables:

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import os
import yaml
from pathlib import Path
from crewai import Agent, Task, Crew

---

### Creating the Agents Definitions

Make an `agents.yaml` file with these agent definitions:

```yaml
feedback_analysis_agent:
  role: >
    Customer Feedback Intelligence Specialist
  goal: >
    Thoroughly analyze customer feedback to identify sentiment nuances, detect recurring issues, and uncover product-specific insights using advanced natural language processing techniques.
  backstory: >
    With extensive experience in text analysis and data mining, you convert raw customer reviews into actionable insights that drive improvements in product quality and customer satisfaction.
  verbose: true
  allow_delegation: false

summary_report_agent:
  role: >
    Report Synthesis Expert
  goal: >
    Consolidate and structure the analyzed feedback into a comprehensive report, highlighting key metrics such as average ratings, sentiment distributions, and emerging trends.
  backstory: >
    You excel at transforming complex datasets into clear, actionable reports that empower stakeholders to make informed decisions.
  verbose: true
  allow_delegation: false

visualization_agent:
  role: >
    Data Visualization Specialist
  goal: >
    Generate intuitive and engaging visualizations using only pie charts rendered in Markdown-compatible Mermaid syntax, to illustrate sentiment breakdowns and product category engagement.
  backstory: >
    Your expertise in data visualization enables you to convert analytical findings into compelling, easily digestible graphics that integrate seamlessly into Markdown reports.
  verbose: true
  allow_delegation: false
```

---

### Creating the Task Definitions

Make a `tasks.yaml` file with these tasks:

```yaml
sentiment_evaluation:
  description: >
    Analyze each customer feedback record to accurately classify the sentiment as Positive, Negative, or Neutral.
    Employ advanced text analytics to extract key themes, identify recurring issues, and capture nuanced sentiment from the feedback text.
  expected_output: >
    A comprehensive sentiment analysis report for each feedback entry, including sentiment labels, key themes, and detected issues.

summary_table_creation:
  description: >
    Aggregate critical metrics from the customer feedback data by creating summary tables.
    Calculate average ratings, count feedback entries per product category, and summarize overall sentiment distributions.
  expected_output: >
    Well-organized tables presenting aggregated customer feedback metrics, prepared for visualization and reporting.

chart_visualization:
  description: >
    Generate visual representations of the summary data using exclusively pie charts.
    Utilize Mermaid syntax to create Markdown-compatible pie charts that display:
    - Sentiment Distribution: The proportion of Positive, Neutral, and Negative feedback.
    - Product Category Distribution: The share of feedback entries per product category.
    Export the charts as Mermaid code snippets ready for embedding in Markdown documents.
  expected_output: >
    A collection of Mermaid-formatted pie charts that clearly visualize the key customer feedback metrics.

final_report_assembly:
  description: >
    Integrate the sentiment analysis, summary tables, and Mermaid-generated pie charts into a cohesive final report.
    The report should deliver clear insights, actionable recommendations, and highlight key trends in customer feedback.
    Ensure that the final document is formatted for easy stakeholder consumption with embedded Markdown visualizations.
  expected_output: >
    A well-structured final report that seamlessly combines data analysis, visualizations, and actionable insights to guide decision-makers.
```

---

### Loading Agents and Tasks definitions

In [ ]:
def load_configurations(file_paths: dict) -> dict:
    """
    Load YAML configurations from the specified file paths.

    Args:
        file_paths (dict): A dictionary mapping configuration names to file paths.

    Returns:
        dict: A dictionary with the loaded YAML configurations.
    Raises:
        FileNotFoundError: If any configuration file is not found.
        yaml.YAMLError: If an error occurs during YAML parsing.
    """
    configs = {}
    for config_type, file_path in file_paths.items():
        path = Path(file_path)
        if not path.exists():
            raise FileNotFoundError(f"Configuration file '{file_path}' for '{config_type}' not found.")
        try:
            configs[config_type] = yaml.safe_load(path.read_text())
        except yaml.YAMLError as error:
            raise yaml.YAMLError(f"Error parsing YAML file '{file_path}': {error}")
    return configs

# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations and assign to variables
configs = load_configurations(files)
agents_config = configs.get('agents')
tasks_config = configs.get('tasks')

---

### Creating the Tools

For reading feedback data:

In [ ]:
from crewai_tools import FileReadTool

csv_tool = FileReadTool(file_path='data/customer_feedback.csv')

You will need a `customer_feedback.csv` file. It should contain columns like:

In [ ]:
feedback_id,customer_id,product_id,product_category,feedback_text,rating,submission_date,response_time_minutes,agent_notes

Sample rows might look like:

In [ ]:
F0001,C101,P1001,Electronics,"The sound quality of the speakers is excellent, but connectivity drops intermittently.",4,2023-03-10,30,"Investigate intermittent connectivity issues."
F0002,C102,P1002,Home,"I loved the modern design of the lamp, although it feels a bit too fragile.",3,2023-03-11,45,"Assess material durability."
...

---

### Creating Agents, Tasks and Crew

In [ ]:
# Creating Agents for Customer Feedback Analysis
feedback_analysis_agent = Agent(
  config=agents_config['feedback_analysis_agent'],
  tools=[csv_tool]
)

summary_report_agent = Agent(
  config=agents_config['summary_report_agent'],
  tools=[csv_tool]
)

visualization_agent = Agent(
  config=agents_config['visualization_agent'],
  allow_code_execution=False
)

# Creating Tasks
sentiment_evaluation = Task(
  config=tasks_config['sentiment_evaluation'],
  agent=feedback_analysis_agent
)

summary_table_creation = Task(
  config=tasks_config['summary_table_creation'],
  agent=summary_report_agent
)

chart_visualization = Task(
  config=tasks_config['chart_visualization'],
  agent=visualization_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=summary_report_agent,
  context=[sentiment_evaluation, summary_table_creation, chart_visualization]
)

# Creating the Customer Feedback Crew
customer_feedback_crew = Crew(
  agents=[
    feedback_analysis_agent,
    summary_report_agent,
    visualization_agent
  ],
  tasks=[
    sentiment_evaluation,
    summary_table_creation,
    chart_visualization,
    final_report_assembly
  ],
  verbose=True
)

---

### Kicking off your Customer Feedback Analyst Agent

In [ ]:
result = customer_feedback_crew.kickoff()

---

## Conclusion

You now have a clear example of how to use CrewAI to build a customer feedback analysis system. You defined your agents and tasks in YAML. You integrated a tool to read the CSV feedback data. You set up a workflow that moves from sentiment evaluation to report summarization to visualization and then final report assembly. The arrangement lets each part reference earlier outputs. Configuration lives outside code so you can version control it or adjust it later.

For those looking to compare this approach with other agent architectures, you might find our tutorial on [building an LLM agent from scratch with GPT\-4 ReAct](https://thegenairevolution.com/blog/44830763/how-to-build-an-llm-agent-from-scratch-with-gpt-4-react-2) insightful, as it covers tool actions and control loops in detail.

You may want to extend this by enabling training mode so you can correct agents iteratively. Or try allowing parallel or hybrid process types if some tasks do not depend on earlier ones. If you want to go further, consider exploring structured data extraction pipelines with external tools or compare with ReAct\-style agents trained from scratch for different trade\-offs.

---

If you like, I can also regenerate your version with more examples, visuals, or test data included.